In [ ]:
import time
import os
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [ ]:
# Reading env vars
SPOTIFY_CLIENT_ID = os.environ.get("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = os.environ.get("SPOTIFY_CLIENT_SECRET")
SPOTIFY_REDIRECT_URI = os.environ.get("SPOTIFY_REDIRECT_URI")
SCOPES = ["user-library-read"]

In [ ]:
def authenticate_spotify(client_id, client_secret, redirect_uri, scope):
    """Authenticate with Spotify using Spotipy"""
    sp_oauth = SpotifyOAuth(
        client_id=SPOTIFY_CLIENT_ID,
        client_secret=SPOTIFY_CLIENT_SECRET,
        redirect_uri=SPOTIFY_REDIRECT_URI,
        scope=SCOPES,
    )

    auth_url = sp_oauth.get_authorize_url()
    print(f"Please go to this URL to authorize the application: {auth_url}")

    code = input("Please enter the code: ")
    token_info = sp_oauth.get_access_token(code)
    access_token = token_info["access_token"]
    return spotipy.Spotify(auth=access_token)

In [ ]:
def get_saved_tracks(sp):
    """Get the user's saved tracks from Spotify"""
    saved_tracks = []
    results = sp.current_user_saved_tracks()
    while results:
        saved_tracks += results["items"]
        if results["next"]:
            results = sp.next(results)
        else:
            results = None
    return saved_tracks

In [ ]:
def get_audio_features(sp, saved_tracks):
    """Get the audio features of the saved tracks"""
    audio_features = []
    for i in range(0, len(saved_tracks), 50):
        track_ids = [
            saved_tracks[j]["track"]["id"]
            for j in range(i, min(i + 50, len(saved_tracks)))
        ]
        audio_features += sp.audio_features(track_ids)
        time.sleep(5)
    return audio_features

In [ ]:
def create_dataframe(saved_tracks, audio_features):
    """Create a Pandas DataFrame with relevant information"""
    df = pd.DataFrame(
        {
            "artist": [track["track"]["artists"][0]["name"] for track in saved_tracks],
            "track_name": [track["track"]["name"] for track in saved_tracks],
            "album_name": [track["track"]["album"]["name"] for track in saved_tracks],
            "popularity": [track["track"]["popularity"] for track in saved_tracks],
            "release_date": [
                track["track"]["album"]["release_date"] for track in saved_tracks
            ],
            "duration_ms": [track["track"]["duration_ms"] for track in saved_tracks],
            "acousticness": [feature["acousticness"] for feature in audio_features],
            "danceability": [feature["danceability"] for feature in audio_features],
            "energy": [feature["energy"] for feature in audio_features],
            "instrumentalness": [
                feature["instrumentalness"] for feature in audio_features
            ],
            "key": [feature["key"] for feature in audio_features],
            "liveness": [feature["liveness"] for feature in audio_features],
            "loudness": [feature["loudness"] for feature in audio_features],
            "mode": [feature["mode"] for feature in audio_features],
            "speechiness": [feature["speechiness"] for feature in audio_features],
            "tempo": [feature["tempo"] for feature in audio_features],
            "time_signature": [feature["time_signature"] for feature in audio_features],
            "valence": [feature["valence"] for feature in audio_features],
        }
    )
    return df

In [ ]:
def save_to_csv(df, file_name):
    """
    Saves the given DataFrame to a CSV file with the given file name.
    """
    df.to_csv(file_name, index=False, encoding="utf-8")
    print(f"DataFrame saved to {file_name}.")

In [ ]:
sp = authenticate_spotify(
    SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET, SPOTIFY_REDIRECT_URI, SCOPES
)

In [ ]:
saved_tracks = get_saved_tracks(sp)

In [ ]:
audio_features = get_audio_features(sp, saved_tracks)

In [ ]:
df = create_dataframe(saved_tracks, audio_features)

In [ ]:
save_to_csv(df, "spotify_playlist.csv")